In [3]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as T
import os
import sys
import argparse
import numpy as np
import torch
from torchvision import transforms, datasets
import torch.utils.data as data
from networks.DDAM_ABAW import DDAMNet
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import random
import torch.nn as nn
from sklearn.metrics import f1_score
from tqdm import tqdm

In [4]:
# Load pre-trained model
model = DDAMNet(num_class=8, num_head=2, pretrained=False, train_val_arousal=True, train_emotions=False, train_actions=False)
model_path = 'checkpoints_ver2.0/affecnet8_epoch25_acc0.6469.pth'
checkpoint = torch.load(model_path, map_location="cpu")  # Load in CPU first to avoid GPU memory spike
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model.eval()
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [6]:
# Define paths
root_dir = "../8th_ABAW"  # Replace with actual path (batch1, batch2)
label_dir = "6th_ABAW_Annotations"  # Replace with actual path

# Define transformations (resize, normalize, convert to tensor)
transform = T.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])]) 

In [7]:
import os
import torch
import torchvision.transforms as transforms
import pickle
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
def process_video(video_name, label_file_path, output_dir, model, root_dir, challenge, device, transform):
    try:
        # Locate the video folder in batch1 or batch2
        video_folder = None
        for batch in ['batch1', 'batch2']:
            possible_path = os.path.join(root_dir, batch, 'cropped_aligned', video_name)
            if os.path.exists(possible_path):
                video_folder = possible_path
                break

        if video_folder is None or not os.path.exists(video_folder):
            print(f"Skipping {video_name}: video folder not found.")
            return

        # Read label file
        with open(label_file_path, 'r') as file:
            lines = file.readlines()[1:]  # Skip header row

        # Get sorted list of frame paths
        frame_paths = sorted([os.path.join(video_folder, frame) for frame in os.listdir(video_folder) if frame.endswith('.jpg')])

        # Process frames in batches
        batch_images = []
        batch_labels = []

        # Output file path
        output_file = os.path.join(output_dir, f"{video_name}.txt")
        with open(output_file, "w") as f:
            for frame_path, label in zip(frame_paths, lines):
                frame_label = list(map(float, label.strip().split(',')))

                # Load and preprocess the image
                image = Image.open(frame_path).convert('RGB')
                image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

                # Append image and label to the batch
                batch_images.append(image)
                batch_labels.append(frame_label)

                # Process the batch if it's full
                if len(batch_images) == 32:
                    # Stack the batch together
                    batch_images_tensor = torch.cat(batch_images, dim=0)

                    # Extract features
                    with torch.no_grad():
                        features = model(batch_images_tensor)  # Shape: (batch_size, 512, 7, 7)

                    # Check if features are a tuple
                    if isinstance(features, tuple):
                        features = features[3]  # Extract the first element (assuming it's the features)

                    # Process each frame in the batch and write to file
                    for i in range(len(batch_images)):
                        feature_vector = features[i].flatten().cpu().numpy()  # Flatten the feature map to a vector
                        label_vector = batch_labels[i]

                        # Write features and labels as a single line in the txt file
                        line = ','.join(map(str, feature_vector)) + ',' + ','.join(map(str, label_vector)) + '\n'
                        f.write(line)

                    # Clear the batch for the next set of images
                    batch_images = []
                    batch_labels = []

            # If there are any remaining frames after the loop
            if len(batch_images) > 0:
                batch_images_tensor = torch.cat(batch_images, dim=0)
                with torch.no_grad():
                    features = model(batch_images_tensor)

                if isinstance(features, tuple):
                    features = features[3]  # Extract the first element (assuming it's the features)

                for i in range(len(batch_images)):
                    feature_vector = features[i].flatten().cpu().numpy()
                    label_vector = batch_labels[i]

                    # Write remaining features and labels to file
                    line = ','.join(map(str, feature_vector)) + ',' + ','.join(map(str, label_vector)) + '\n'
                    f.write(line)

        print(f"Processed {video_name} -> {output_file}")

    except Exception as e:
        print(f"Error processing {video_name}: {str(e)}")

def extract_and_save_features(model, root_dir, label_dir, challenge, output_root):
    """
    Extracts features from video frames and saves them as Pickle (.pkl) files.
    """
    # Validate challenge type
    challenge_folders = {'AU': 'AU_Detection_Challenge', 'EXPR': 'EXPR_Recognition_Challenge', 'VA': 'VA_Estimation_Challenge'}
    if challenge not in challenge_folders:
        raise ValueError(f"Invalid challenge: {challenge}. Must be 'AU', 'EXPR', or 'VA'.")

    challenge_folder = challenge_folders[challenge]

    # Define label folder paths
    train_label_folder = os.path.join(label_dir, challenge_folder, 'Train_Set')
    val_label_folder = os.path.join(label_dir, challenge_folder, 'Validation_Set')

    # Define output directories
    output_challenge_dir = os.path.join(output_root, challenge)  # e.g., output_root/AU/
    train_output_dir = os.path.join(output_challenge_dir, 'training_set_features')
    val_output_dir = os.path.join(output_challenge_dir, 'validation_set_features')

    # Create directories if they don't exist
    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    # Find label files
    label_files = []
    for folder, output_dir in [(train_label_folder, train_output_dir), (val_label_folder, val_output_dir)]:
        if os.path.exists(folder):
            label_files += [(folder, output_dir, f) for f in os.listdir(folder) if f.endswith('.txt')]

    if not label_files:
        raise FileNotFoundError(f"No label files found in {train_label_folder} or {val_label_folder}.")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Handle outliers based on challenge type
    if challenge == 'VA':
        model = DDAMNet(num_class=8, num_head=2, pretrained=False, train_val_arousal=True, train_emotions=False, train_actions=False)
        print("Model initialized for VA challenge")

    if challenge == 'EXPR':
        model = DDAMNet(num_class=8, num_head=2, pretrained=False, train_val_arousal=False, train_emotions=True, train_actions=False)
        print("Model initialized for EXPR challenge")

    if challenge == 'AU':
        model = DDAMNet(num_class=8, num_head=2, pretrained=False, train_val_arousal=False, train_emotions=False, train_actions=True)
        print("Model initialized for AU challenge")
    model_path = 'checkpoints_ver2.0/affecnet8_epoch25_acc0.6469.pth'
    checkpoint = torch.load(model_path, map_location="cpu")  # Load in CPU first to avoid GPU memory spike
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Define transformations (resize, normalize, convert to tensor)
    transform = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    total_videos = len(label_files)
    with ThreadPoolExecutor() as executor:
        futures = []
        with tqdm(total=total_videos, desc="Processing videos") as pbar:
            for label_folder, output_dir, label_file in label_files:
                video_name = label_file.replace('.txt', '')
                label_file_path = os.path.join(label_folder, label_file)
                futures.append(executor.submit(process_video, video_name, label_file_path, output_dir, model, root_dir, challenge, device, transform))

            # Wait for all futures to complete
            for future in futures:
                future.result()

    print(f"Feature extraction completed for {challenge}. Files saved in {output_challenge_dir}")


In [8]:
challenges = ["VA","EXPR","AU"]    
for i in challenges:
    extract_and_save_features(
        model=model,  # Your pre-trained model
        root_dir=root_dir,  # Root dataset directory
        label_dir=label_dir,  # Label directory
        challenge=i,  # 'AU', 'EXPR', or 'VA'
        output_root="Features"  # Where to save extracted features
    )
